In [1]:
################################################################################################################################
######################################## PART I: Transforming Data Into Level Files ############################################
################################################################################################################################

In [2]:
# Libraries to manipulate DataFrames
import pandas as pd

# Library which contains some useful functions
import numpy as np

# To hide useless warnings
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML

In [3]:
# Read org data
jnjOrgData = pd.read_excel('J&J Org Data (Aug 2021, C-7 Levels, With Notes).xlsx', encoding='latin-1')
jnjOrgData['WWID'] = jnjOrgData['WWID'].astype(int).astype(str)
jnjOrgData['Supervisor WWID'] = jnjOrgData['Supervisor WWID'].astype(int).astype(str)
jnjOrgData.fillna('', inplace=True)

In [4]:
# Create a list of dataframes for each level dataframe to be appended to
dataframeList = []

# fetch ID which got assigned to Corbat, Michael L
ceoWWID = jnjOrgData[jnjOrgData['Employee Name'].str.contains('Gorsky')]['WWID'].iloc[0]

# Initialize a counter to keep track of levels
currentLevelCounter = 0

# Creating a separate dataframe and level file for CEO with all his details
# This "Level 0" file will be different from other level files in regard that CEO would have no supervisor
levelZeroDataframe = pd.DataFrame()
ceoName = jnjOrgData[jnjOrgData['Employee Name'].str.contains('Gorsky')]['Employee Name'].iloc[0]
ceoEmail = jnjOrgData[jnjOrgData['WWID'] == ceoWWID]['Email Address'].iloc[0]
ceoPhone = jnjOrgData[jnjOrgData['WWID'] == ceoWWID]['Business Telephone'].iloc[0]
ceoTitle = jnjOrgData[jnjOrgData['WWID'] == ceoWWID]['Title'].iloc[0]
ceoAddress = jnjOrgData[jnjOrgData['WWID'] == ceoWWID]['Business Address'].iloc[0]
ceoDepartment = jnjOrgData[jnjOrgData['WWID'] == ceoWWID]['Department'].iloc[0]

levelZeroDataframe = levelZeroDataframe.append({'WWID':ceoWWID, 
                                                'Email Address':ceoEmail, 
                                                'Business Telephone':ceoPhone, 
                                                'Title':ceoTitle, 
                                                'Business Address':ceoAddress, 
                                                'Employee Name':ceoName, 
                                                'Department':ceoDepartment,
                                                'Company Name':ceoDepartment,
                                                'Partner Company Name':np.nan}, ignore_index=True)
        
# Write out CEO level file
levelZeroDataframe.to_excel('Level 0.xlsx', index=False)

# Append Level 0 dataframe to dataframe list
dataframeList.append(levelZeroDataframe.fillna(''))

# Remove row corresponding to CEO as CEO has no supervisor and can cause errors in the following code 
jnjOrgData.drop(jnjOrgData[jnjOrgData['WWID'] == ceoWWID].index, inplace=True)

# Create a function to process data level by level
def parseOrgData(supervisorList):
    global currentLevelCounter
    
    # Incrementing level counter as each level is processed
    currentLevelCounter = currentLevelCounter + 1
    
    # Get data of all those employees which report to supervisors present at a level
    # For example this dataframe would contain data of Level 3 employees would who be reporting to Level 2 supervisors
    currentLevelDataFrame = jnjOrgData[jnjOrgData['Supervisor WWID'].isin(supervisorList)]
        
    # If at least one reportee is found for any of the supervisors sitting at a given level
    if currentLevelDataFrame.shape[0] > 0:
        
        # Reordering columns
        currentLevelDataFrame = currentLevelDataFrame[['Supervisor WWID', 
                                                       'Employee Name',
                                                       'WWID', 
                                                       'Email Address', 
                                                       'Business Telephone',
                                                       'Title', 
                                                       'Business Address', 
                                                       'Department', 
                                                       'Company Name', 
                                                       'Partner Company Name']]
        
        # Writing out data file corresponding to current level
        currentLevelDataFrame.to_excel('Level ' + str(currentLevelCounter) + '.xlsx', index=False)
        
        # Appending each level dataframe to dataframe list
        dataframeList.append(currentLevelDataFrame.fillna(''))
        
        print('Processed Level ' + str(currentLevelCounter) + ' data\n')
        
        # Now current level's subordinates would become next level's supervisors
        # Getting those employees' WWIDs and recursively calling the function
        parseOrgData(list(currentLevelDataFrame['WWID'].drop_duplicates()))

parseOrgData([ceoWWID])

Processed Level 1 data

Processed Level 2 data

Processed Level 3 data

Processed Level 4 data

Processed Level 5 data

Processed Level 6 data

Processed Level 7 data

Processed Level 8 data



In [5]:
################################################################################################################################
######################################### PART II: Transforming Level Files Into JSON Data #####################################
################################################################################################################################

In [6]:
import os.path

# Library to write out dictionary in JSON format
import json

# A library for some cool progress bars
from tqdm import tqdm
# Initialize Progress bar library
tqdm.pandas()

# For clearing output if needed
from IPython.display import clear_output

In [7]:
# Create an empty list which will hold dictionaries for every level
dictionaryList = []

# Iterate over every DataFrame present in `dataframeList`
for dataframeIndex, dataframe in enumerate(tqdm(dataframeList)):
    # Initialize an empty dictionary for the level which is being iterated upon
    levelDictionary = dict()
    # Iterate over every employee record in each DataFrame
    for employeeIndex, employee in dataframe.iterrows():
        # Initialize an empty list which will hold the required attributes corresponding to every employee we iterate upon
        record = []
        # Skipping `Supervisor ID` attribute for level 0 employee which is the CEO himself
        if dataframeIndex != 0:
            record += ['supervisorWWID^' + str(employee['Supervisor WWID'])]
        # Appending required attributes to `record` list after type conversion to string format
        record += ['name^' + str(employee['Employee Name'])]
        record += ['wwid^' + str(employee['WWID'])]
        record += ['companyName^' + str(employee['Company Name'])]
        record += ['businessTelephone^' + str(employee['Business Telephone'])]
        record += ['emailAddress^' + str(employee['Email Address'])]
        record += ['title^' + str(employee['Title'])]
        record += ['businessAddress^' + str(employee['Business Address'])]
        record += ['department^' + str(employee['Department'])]
        record += ['partnerCompanyName^' + str(employee['Partner Company Name'])]
        # Append record to level dictionary
        levelDictionary.update({employeeIndex:dict(item.split("^",1) for item in record)})
    # When all records are processed on a level, append level dictionary to master dictionary list initialized above
    dictionaryList.append(levelDictionary)

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:25<00:00,  2.86s/it]


In [8]:
# Initialize `children` key
# This is needed as d3.js code requires org tree data in such format
key = 'children'

# Iterate over every level of `dictionaryList` populated above
# Starting from the lowest level, i.e., level 5 or 6
for dictionaryCounter in tqdm(range (len(dictionaryList)-1, 0, -1)):
    # Iterate over every subordinate employee record on a level
    for subordinateKey, subordinateValue in dictionaryList[dictionaryCounter].items():
        # Iterate over every superior employee who sits on one level above subordinates
        for superiorKey, superiorValue in dictionaryList[dictionaryCounter - 1].items():
            # Match `Subordinate's SUPERVISOR ID` with `Superior's ID`
            if subordinateValue['supervisorWWID'] == superiorValue['wwid']:
                # In case of positive match, append subordinate's data as superior's 'children' attribute
                dictionaryList[dictionaryCounter - 1][superiorKey].setdefault(key, [])
                dictionaryList[dictionaryCounter - 1][superiorKey][key].append(dictionaryList[dictionaryCounter][subordinateKey])

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [13:18<00:00, 99.80s/it]


In [9]:
# Write out created dictionary to json file
jsonFile = open("J&J Org Chart Data.json", "w")
# Reason why index is given to be 0,0 is because upon backtracking from the lowest level
# and appending data to higher levels finally results in all org data to be appended as CEO's children 
# who sits at level 0, in a hierarchial fashion
jsonFile.write(json.dumps(dictionaryList[0][0]))
jsonFile.close()

In [10]:
# Defining a function for injecting JSON 
def injectJSON(jsonData, levelCount):
    
    # Specifying raw HTML code path
    htmlCodeFilePath = 'Org Tree Raw HTML Code.txt'
    # Specifying HTML Page's preferred title (optional)
    htmlPageTitle = 'J&J Org Tree (Aug 2021, C-7 Levels)'
    
    # Checking if raw HTML file exists, if it does, read it, line by line
    if os.path.isfile(htmlCodeFilePath):
        with open(htmlCodeFilePath, "r") as htmlCodeFile:
            htmlCode = "".join(htmlCodeFile.readlines()[1:])
            
        # Inject Web Page Title
        htmlCode = htmlCode.replace('<Title></Title>', '<Title>' + htmlPageTitle + '</Title>')
        # Inject JSON data 
        htmlCode = htmlCode.replace('root = [];', 'root = [' + jsonData + '];')
        
        # Write injected data out to HTML file
        with open(htmlPageTitle + ".html", "w") as text_file:
            text_file.write(htmlCode)
        
        return 'JSON successfully injected in HTML Code and saved!!!'
    else:
        return 'Raw HTML code not found!!!'

In [11]:
# Call JSON injector function with JSON data and number of levels as arguments
injectJSON(json.dumps(dictionaryList[0][0]), str(currentLevelCounter-1))

'JSON successfully injected in HTML Code and saved!!!'